In [1]:
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data,DataLoader
from functions import *
from pytorch_util import *
from torch.optim import Adam
from torch.nn.utils import clip_grad_value_
from torch_geometric.nn import NNConv,GATConv

In [2]:
with open('../Data/train_data_ACSF.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('../Data/val_data_ACSF.pickle', 'rb') as handle:
    val_data = pickle.load(handle)
with open('../Data/test_data_ACSF.pickle', 'rb') as handle:
    test_data = pickle.load(handle)

In [3]:
### parameters ###
batch_size = 32
dim = 64
edge_dim = 12
epochs = 5
clip = 2

### load dataset

In [4]:
train_list = [Data(**d) for d in train_data]
train_dl = DataLoader(train_list,batch_size,shuffle=True)

In [5]:
val_list = [Data(**d) for d in val_data]
valid_dl = DataLoader(val_list,batch_size,shuffle=False)

In [6]:
test_list = [Data(**d) for d in test_data]
test_dl = DataLoader(test_list,batch_size,shuffle=False)

In [20]:
class Net_int_2Edges_attention2(torch.nn.Module):
    # use both types of edges
    def __init__(self,dim=64,edge_dim=12,node_in=8,edge_in=19,edge_in3=8):
        super(Net_int_2Edges_attention2, self).__init__()
        self.lin_node = torch.nn.Linear(node_in, dim)
        
        self.conv1 = GATConv(dim, dim, negative_slope=0.2, bias=True)
        
        self.lin_covert1 = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim),RReLU())     
        
        self.conv2 = GATConv(dim, dim, negative_slope=0.2, bias=True)
        
        self.lin_covert2 = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim),RReLU())     
        
        self.conv3 = GATConv(dim, dim, negative_slope=0.2, bias=True)
        
        self.lin_covert3 = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                             RReLU(), Dropout(),Linear(dim*2, dim),RReLU())
        
        self.conv4 = GATConv(dim, dim, negative_slope=0.2, bias=True)
        
        self.lin_covert4 = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                             RReLU(), Dropout(),Linear(dim*2, dim),RReLU())
        
        self.lin_weight = Linear(8, dim*3, bias=False)
        self.lin_bias = Linear(8, 1, bias=False)
        self.norm = BatchNorm1d(dim*3)
        self.norm_x = BatchNorm1d(node_in)
        
    def forward(self, data,IsTrain=False):
        out = F.rrelu(self.lin_node(self.norm_x(data.x)))
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        
        m = F.rrelu(self.conv1(out, data.edge_index))
        out = out + m
        out = self.lin_covert1(out)
        
        m = F.rrelu(self.conv2(out, data.edge_index))
        out = out + m
        out = self.lin_covert2(out)        
        
        m = F.rrelu(self.conv3(out, edge_index3))
        out = out + m
        out = self.lin_covert3(out)          

        m = F.rrelu(self.conv4(out, edge_index3))
        out = out + m
        out = self.lin_covert4(out)
            
        temp = out[data.edge_index3] # (2,N,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)
        yhat = self.norm(yhat)
        weight = self.lin_weight(data.edge_attr3)
        bias = self.lin_bias(data.edge_attr3)
        yhat = torch.sum(yhat * weight,1,keepdim=True) + bias
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat    

### build model and set up training

In [21]:
model = Net_int_2Edges_attention2(dim=dim,edge_dim=edge_dim,node_in=89).to('cuda:0')

In [22]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=3e-4)

In [23]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:1.2791775501259506, val_loss:0.8186060223314497
epoch:1, train_loss:0.8132211950526261, val_loss:0.5349225143973644
epoch:2, train_loss:0.6052494417138442, val_loss:0.4303735360248476
epoch:3, train_loss:0.5350772178249377, val_loss:0.324781351046175
epoch:4, train_loss:0.4879612693734293, val_loss:0.28944975492727554
Training completed in 183.21138310432434s


In [24]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.4574440883946953, val_loss:0.26612492762187606
epoch:1, train_loss:0.43207789612078046, val_loss:0.2348975665294207
epoch:2, train_loss:0.41075278460216347, val_loss:0.2105004230243528
epoch:3, train_loss:0.391977055743507, val_loss:0.1823328783353552
epoch:4, train_loss:0.378984748629004, val_loss:0.184809929301214
Training completed in 183.8660340309143s


In [25]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.36182988392445503, val_loss:0.15511146343002716
epoch:1, train_loss:0.35026735547039134, val_loss:0.17429589773488477
epoch:2, train_loss:0.3376431537508474, val_loss:0.14438349737698197
epoch:3, train_loss:0.3292958297050048, val_loss:0.13801290054256335
epoch:4, train_loss:0.31908704831712476, val_loss:0.10966746275050518
Training completed in 183.49691915512085s


In [26]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.31180939777397815, val_loss:0.1034011408392117
epoch:1, train_loss:0.3029526636229294, val_loss:0.10482341851879898
epoch:2, train_loss:0.2931707252275197, val_loss:0.08015800026667297
epoch:3, train_loss:0.28849952991252953, val_loss:0.07869292315668784
epoch:4, train_loss:0.28175506480021134, val_loss:0.06655212541102853
Training completed in 183.91949582099915s


In [45]:
checkpoint = torch.load('../Model/gnn_int_logloss_2Edges.tar')
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

In [109]:
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, '../Model/gnn_int_logloss_2Edges_ACSF.tar')

### make submissions

In [110]:
model.eval()
yhat_list = []
with torch.no_grad():
    for data in test_dl:
        data = data.to('cuda:0')
        yhat_list.append(model(data,False))

yhat = torch.cat(yhat_list).cpu().detach().numpy()
submission = pd.read_csv('../Data/sample_submission.csv')
submission['scalar_coupling_constant'] = yhat
submission.to_csv('../Submission/gnn_int_logloss_2Edges_ACSF.csv',index=False)